### Import the libraries

In [ ]:
from astropy.time import Time
from astroquery.jplhorizons import Horizons

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import requests
import time

In [ ]:
# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

---------------------------------------------------------------------------------------

## Read the catalog

In [ ]:
# Dataframe
df = pd.read_csv('SDSS-data.csv', header=None, usecols=[7,19,20,21,22,23,24,25,26,27,28,34,35,42,43,44], dtype={34: 'string'})
df = df.rename(columns={7:'MJD', 19: 'u', 20: 'err_u', 21: 'g', 22: 'err_g', 23: 'r-mag', 24: 'err_r', 25: 'i', 26: 'err_i', 27: 'z', 28: 'err_z', 34: 'id', 35: 'des', 
                       42: 'r', 43: 'delta', 44: 'alpha'})

In [ ]:
# Make the designation readable for Horizons
df.replace('0', np.nan, inplace=True)
df['id'].fillna(df['des'], inplace=True)
df = df.replace('_', ' ', regex=True)
df = df[~df.apply(lambda row: row.astype(str).str.contains('-').any(), axis=1)]

### Avoid repeating asteorids

Sometimes, during the Horizons query, some problems may appear. For example, bad internet connection. If we restart the code,
we don't want to repeat the process for asteroids that already have a .csv file. This part of the code prevent that.

In [ ]:
# Specify the folder path where your CSV files are located
folder_path = '/home/milagros/Documents/SDSS-files'

# Initialize an empty list to store the formatted asteroid numbers
formatted_asteroid_numbers = []

# Define a regular expression pattern to match the filenames
pattern = r'SDSS-asteroid([\w\s]+)\.csv'

# List all files in the folder
files = os.listdir(folder_path)

# Iterate through the files and extract the numbers
for filename in files:
    match = re.match(pattern, filename)
    if match:
        value = match.group(1)
        formatted_asteroid_numbers.append(value)

In [ ]:
df = df[~df['id'].isin(formatted_asteroid_numbers)]

In [ ]:
# This objects are not in Horizons system
df = df[(df['id'] != '2002 TN365') & (df['id'] != '2002 UB67') & (df['id'] != '2002 UZ67') & (df['id'] != '2005 VL128')] 

In [ ]:
df = df.sort_values(by=['id'])

In [ ]:
# Each magnitude has its column but we need only one magnitude column, then we use another column to specify the filter
df_1 = df[['id', 'MJD', 'u', 'err_u', 'r', 'delta', 'alpha']]
df_2 = df[['id', 'MJD', 'g', 'err_g', 'r', 'delta', 'alpha']]
df_3 = df[['id', 'MJD', 'r-mag', 'err_r', 'r', 'delta', 'alpha']]
df_4 = df[['id', 'MJD', 'i', 'err_i', 'r', 'delta', 'alpha']]
df_5 = df[['id', 'MJD', 'z', 'err_z', 'r', 'delta', 'alpha']]

In [ ]:
# Fix column names and adding the filter column
df_1 = df_1.rename(columns={'u':'m', 'err_u': 'err_m'})
df_1.insert(7, 'Filter', 'u')

df_2 = df_2.rename(columns={'g':'m', 'err_g': 'err_m'})
df_2.insert(7, 'Filter', 'g')

df_3 = df_3.rename(columns={'r-mag':'m', 'err_r': 'err_m'})
df_3.insert(7, 'Filter', 'r')

df_4 = df_4.rename(columns={'i':'m', 'err_i': 'err_m'})
df_4.insert(7, 'Filter', 'i')

df_5 = df_5.rename(columns={'z':'m', 'err_z': 'err_m'})
df_5.insert(7, 'Filter', 'z')

In [ ]:
# Merge everything
pdList = [df_1, df_2, df_3, df_4, df_5]  # List of your dataframes
df = pd.concat(pdList)

In [ ]:
df = df.sort_values(by='id')

In [ ]:
df = df.rename(columns={'id':'idAsteroid'})

In [ ]:
df = df[['idAsteroid', 'MJD', 'm', 'err_m', 'alpha','Filter', 'delta', 'r']]

In [ ]:
# We need to perform some calculations to obtain the desired format for the table.

# Compute reduced magnitude
m_red = df['m'] - 5*np.log10(df['r']*df['delta'])
m_red = m_red.to_numpy()
df.insert(5, "m_red", m_red, True)
# ----------------------------------------------------------------------------------------------------------------------
# Compute flux
flux = 10**(-0.4*df.m)
flux = flux.to_numpy()
df.insert(4, "flux", flux, True)
e_flux = 0.4*10**(-0.4*df.m)*df.err_m
df.insert(5, "err_flux", e_flux, True)
# ----------------------------------------------------------------------------------------------------------------------
# Compute JD
jd = df['MJD'] + 2400000.5
df.insert(2, "jd", jd, True)
# ----------------------------------------------------------------------------------------------------------------------
#Add ephemeris columns
#df.insert(1, 'Desig', '')
df.insert(6, 'ElongFlag', '')
# ----------------------------------------------------------------------------------------------------------------------
#Add constant columns
df.insert(7, 'Source', 'SDSS')
df.insert(8, 'TypePhotometry', 'relative')
df.insert(9, 'LTcorrected', 1)
# ----------------------------------------------------------------------------------------------------------------------

## Splitting dataframe

We have a dataframe with all the asteroids. For the ephemeris step, we need to proceed asteroid by asteroid.

In [ ]:
# Splitting dataframes
df = df[['idAsteroid', 'jd', 'm', 'err_m', 'flux', 'err_flux', 'm_red','Filter', 'alpha', 'delta', 'r', 'ElongFlag', 'Source', 'TypePhotometry', 'LTcorrected']]
sample = df
ids_sample = sample['idAsteroid'].drop_duplicates()
ids_sample = ids_sample.to_numpy()
len = ids_sample.size
print('Splitting...')
# Function to split DataFrame based on the changing value in the 'target_column'
def split_dataframe_on_value_change(df, column_name):
    df_list = []
    prev_value = None
    temp_df = None

    for index, row in df.iterrows():
        current_value = row[column_name]

        if prev_value is None or current_value != prev_value:
            if temp_df is not None:
                df_list.append(temp_df)
            temp_df = pd.DataFrame(columns=df.columns)

        temp_df = pd.concat([temp_df, row.to_frame().T], ignore_index=True)
        prev_value = current_value

    if temp_df is not None:
        df_list.append(temp_df)

    return df_list
# Split the DataFrame based on the changing value in the 'target_column'
resulting_dataframes = split_dataframe_on_value_change(sample, 'idAsteroid')
# ----------------------------------------------------------------------------------------------------------------------
print('Splitted')

## Ephemeris

In [ ]:
# Ephemeris
for idx, df in enumerate(resulting_dataframes):
    ids = df['idAsteroid'].drop_duplicates().to_numpy()[0]
    jd = df['jd'].to_numpy()

    # For many asteroids we have 5 observations (one per filter) with the same Julian Date. Horizons can't work with that.
    # So, we split this step in two: for only one observation and for many observations
    
    if jd.size == 1:

        #Ephemeris query
        e_obj = Horizons(id=ids, location='645', epochs=jd, id_type='smallbody') # Julian date in UTC for ephemerides
        eph = e_obj.ephemerides()
        elong_flag = eph.columns['elongFlag']

        #Vectors query
        jd_tdb = Time(jd.tolist(), format='jd', scale='utc').tdb.value
        v_obj = Horizons(id=ids, location='645', epochs=jd_tdb, id_type='smallbody') # Julian date in TDB for vectors
        vec = v_obj.vectors()
        x_obs = vec.columns['x']
        y_obs = vec.columns['y']
        z_obs = vec.columns['z']
        LT_obs = vec.columns['lighttime']

        #Query Sun-----------------------------------------
        obj_sun = Horizons(id=ids, location='500@10',
                epochs=jd_tdb, id_type='smallbody')
        vec_sun = obj_sun.vectors()
        x_sun = vec_sun.columns['x']
        y_sun = vec_sun.columns['y']
        z_sun = vec_sun.columns['z']  

        #Put in dataframe
        #df.loc[idx,'Desig'] = desig[0]
        df['ElongFlag'] = elong_flag
        df['x_obs'] = x_obs
        df['y_obs'] = y_obs
        df['z_obs'] = z_obs
        df['x_sun'] = x_sun
        df['y_sun'] = y_sun
        df['z_sun'] = z_sun
        df['LT_obs'] = LT_obs
        
        df.to_csv('/home/milagros/Documents/SDSS-files/SDSS-asteroid'+str(ids)+'.csv', index=False)       
        del obj
        del eph
        del obj_sun
        del vec_sun
        gc.collect()
        print(ids, 'small done')
        
    else:
        split_dataframes = np.array_split(df, jd.size)
        calculated_dataframes = []
        for i in split_dataframes:

            #Ephemeris query
            e_obj = Horizons(id=ids, location='645',epochs=i.jd_tdb.to_numpy(), id_type='smallbody')
            eph = e_obj.ephemerides()
            elong_flag = eph.columns['elongFlag']
            
            #Vectors query
            jd_tdb = Time(i.jd.tolist(), format='jd', scale='utc').tdb.value
            v_obj = Horizons(id=ids, location='645',epochs=jd_tdb, id_type='smallbody')
            vec = v_obj.vectors()
            x_obs = vec.columns['x']
            y_obs = vec.columns['y']
            z_obs = vec.columns['z']
            LT_obs = vec.columns['lighttime']
            
            #Query Sun-----------------------------------------
            obj_sun = Horizons(id=ids, location='500@10',epochs=jd_tdb, id_type='smallbody')
            vec_sun = obj_sun.vectors()
            x_sun = vec_sun.columns['x']
            y_sun = vec_sun.columns['y']
            z_sun = vec_sun.columns['z']
            
            i['ElongFlag'] = elong_flag
            i['x_obs'] = x_obs
            i['y_obs'] = y_obs
            i['z_obs'] = z_obs
            i['x_sun'] = x_sun
            i['y_sun'] = y_sun
            i['z_sun'] = z_sun
            i['LT_obs'] = LT_obs

            calculated_dataframes.append(i)
            
        merged_df = pd.concat(calculated_dataframes, ignore_index=True)
        
        merged_df['ElongFlag'] = merged_df['ElongFlag'].to_numpy()
        merged_df['x_obs'] = merged_df['x_obs'].to_numpy()
        merged_df['y_obs'] = merged_df['y_obs'].to_numpy()
        merged_df['z_obs'] = merged_df['z_obs'].to_numpy()
        merged_df['x_sun'] = merged_df['x_sun'].to_numpy()
        merged_df['y_sun'] = merged_df['y_sun'].to_numpy()
        merged_df['z_sun'] = merged_df['z_sun'].to_numpy()
        merged_df['LT_obs'] = merged_df['LT_obs'].to_numpy()
        
        merged_df.to_csv('/home/milagros/Documents/SDSS-files/SDSS-asteroid'+str(ids)+'.csv', index=False)
        print(ids, 'big done')
        del obj
        del eph
        del obj_sun
        del vec_sun
        gc.collect()
        
print('Done')

The end :)